In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\P_126\Dataset_new\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

# Define the CNN model for feature extraction
cnn_input = Input(shape=(img_width, img_height, 3))
cnn_model = Sequential([
    Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten()
])
cnn_output = cnn_model(cnn_input)
cnn_output_reshaped = tf.reshape(cnn_output, (-1, 1, cnn_output.shape[1]))  # Reshape for LSTM

# Define the LSTM model for sequence modeling
lstm_model = Sequential([
    LSTM(128, input_shape=(1, cnn_output.shape[1])),
    Dense(len(class_names), activation='softmax')
])
lstm_output = lstm_model(cnn_output_reshaped)

# Combine the CNN and LSTM models
combined_model = Model(inputs=cnn_input, outputs=lstm_output)

# Compile and train the model
combined_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
combined_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50)
# Save the trained model
combined_model.save('lung_detection_cnnlstm.h5')


Found 9971 images belonging to 6 classes.



Epoch 1/50


312/312 [==============================] - 621s 2s/step - loss: 0.8806 - accuracy: 0.6665
Epoch 2/50
312/312 [==============================] - 588s 2s/step - loss: 0.6207 - accuracy: 0.7753
Epoch 3/50
312/312 [==============================] - 586s 2s/step - loss: 0.5296 - accuracy: 0.8057
Epoch 4/50
312/312 [==============================] - 577s 2s/step - loss: 0.4418 - accuracy: 0.8384
Epoch 5/50
312/312 [==============================] - 583s 2s/step - loss: 0.4152 - accuracy: 0.8486
Epoch 6/50
312/312 [==============================] - 579s 2s/step - loss: 0.3861 - accuracy: 0.8611
Epoch 7/50
312/312 [==============================] - 580s 2s/step - loss: 0.3368 - accuracy: 0.8822
Epoch 8/50
312/312 [==============================] - 580s 2s/step - loss: 0.3240 - accuracy: 0.8828
Epoch 9/50
312/312 [==============================] - 578s 2s/step - loss: 0.3013 - accuracy: 0.8936
Epoch 10/50
312/312 [======================

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
# Set the path to your test dataset
test_data_dir = r"D:\P_126\Dataset_new\test-20230326T155708Z-001\test"

# Create data generator for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',  # Use 'sparse' for integer labels
    shuffle=False)  # Disable shuffling to maintain the order of images

# Evaluate the model on the test data
loss, accuracy = combined_model.evaluate(test_generator)

print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

# Optionally, make predictions on the test data
predictions = combined_model.predict(test_generator)


Found 1966 images belonging to 6 classes.
62/62 [==============================] - 21s 339ms/step - loss: 0.4007 - accuracy: 0.8957
Test loss: 0.4006696343421936
Test accuracy: 0.8957273364067078
62/62 [==============================] - 21s 339ms/step


In [8]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_cnnlstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"C:\Users\Administrator\Desktop\00000403_001.png"
# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
# img_array = np.expand_dims(img_array, axis=1)  # Remove this line
 # Add a batch dimension
 # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")

1/1 [==============================] - 1s 586ms/step
Predicted class: NORMAL


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = r"D:\P_126\Dataset_new\train-20230326T152931Z-001\train"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Extract class names from the folder names
class_names = sorted(os.listdir(train_data_dir))

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')  # Use 'sparse' for integer labels

# Define the CNN model for feature extraction
cnn_input = Input(shape=(img_width, img_height, 3))
cnn_model = Sequential([
    Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten()
])
cnn_output = cnn_model(cnn_input)
cnn_output_reshaped = tf.reshape(cnn_output, (-1, 1, cnn_output.shape[1]))  # Reshape for LSTM

# Define the LSTM model for sequence modeling
lstm_model = Sequential([
    LSTM(128, input_shape=(1, cnn_output.shape[1])),
    Dense(len(class_names), activation='softmax')
])
lstm_output = lstm_model(cnn_output_reshaped)

# Combine the CNN and LSTM models
combined_model = Model(inputs=cnn_input, outputs=lstm_output)

# Compile and train the model
combined_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
combined_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30)
# Save the trained model
combined_model.save('lung_detection_cnnlstm.h5')

Found 9971 images belonging to 6 classes.
Epoch 1/30
312/312 [==============================] - 480s 1s/step - loss: 0.9238 - accuracy: 0.6522
Epoch 2/30
312/312 [==============================] - 341s 1s/step - loss: 0.6136 - accuracy: 0.7760
Epoch 3/30
312/312 [==============================] - 486s 2s/step - loss: 0.5332 - accuracy: 0.8035
Epoch 4/30
312/312 [==============================] - 685s 2s/step - loss: 0.5001 - accuracy: 0.8208
Epoch 5/30
182/312 [================>.............] - ETA: 4:15 - loss: 0.4370 - accuracy: 0.8474

In [ ]:
# Set the path to your test dataset
test_data_dir = r"D:\P_126\Dataset_new\test-20230326T155708Z-001\test"

# Create data generator for test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',  # Use 'sparse' for integer labels
    shuffle=False)  # Disable shuffling to maintain the order of images

# Evaluate the model on the test data
loss, accuracy = combined_model.evaluate(test_generator)

print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

# Optionally, make predictions on the test data
predictions = combined_model.predict(test_generator)


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the saved model
model_path = 'lung_detection_cnnlstm.h5'
loaded_model = load_model(model_path)

# Define your custom class names
class_names = ['CARDIOMEGALY', 'COVID', 'NORMAL', 'PNEUMONIA', 'PNEUMOTHORAX', 'TUBERCULOSIS']

# Path to the image you want to predict
image_path = r"C:\Users\Administrator\Desktop\00000403_001.png"
# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
# img_array = np.expand_dims(img_array, axis=1)  # Remove this line
 # Add a batch dimension
 # Add a time dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make a prediction
predictions = loaded_model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get the predicted class name from the custom class names list
predicted_class_name = class_names[predicted_class]

# Display the predicted class
print(f"Predicted class: {predicted_class_name}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert predicted probabilities into class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels from the test generator
true_labels = test_generator.classes

# Compute precision, recall, and the confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names))

print("Confusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))
